In [57]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import dash_daq as daq


# Load the CSV file into a DataFrame
df = pd.read_csv('Student Profiles Wrangled.csv')

In [58]:
def categorize_gpa(row):
    gpa = row['GPA']
    if gpa < 1:
        return 'Less Than 1'
    elif gpa < 2 and gpa >= 1:
        return '1 - 2'
    elif gpa < 3 and gpa >= 2:
        return '2 - 3'
    elif gpa < 3.5 and gpa >= 3:
        return '3 - 3.5'
    else:
        return 'More Than 3.5'

df['GPA Category'] = df.apply(categorize_gpa, axis=1)

In [59]:
df

,STUDENT ID,SALUTATION,GENDER,NATIONALITY,DOB,HIGHEST QUALIFICATION,NAME OF QUALIFICATION AND INSTITUTION,DATE ATTAINED HIGHEST QUALIFICATION,DESIGNATION,INTAKE NO,...,COURSE FUNDING,REGISTRATION FEE,PAYMENT MODE,COURSE FEE,GPA,CITIZENSHIP_STATUS,date_diff,COURSE,Age,GPA Category
0,2020/1101-013/001,Ms,F,Singaporean,1978-04-03,Certificate,"Certificate in Office Skills, ITE",2016-11-06,"Snr Associate, Client Services",13th,...,Subsidiesed,107,NETS,1712,3.0,L,456,1101,46,3 - 3.5
1,2020/1101-013/002,Ms,F,Singaporean,1966-03-23,Certificate,"WSQ Higher Certificate in Human Resources, WPL...",2018-02-06,Admin Officer,13th,...,Individual,107,NETS,1712,2.5,L,456,1101,58,2 - 3
2,2020/1101-013/003,Ms,F,Singaporean,1988-04-12,Certificate,"Nitec in Service Skills (Office), ITE",2016-11-06,Admin Assistant,13th,...,Sponsored,107,Bank Transfer,1712,2.0,L,456,1101,36,2 - 3
3,2020/1101-014/004,Ms,F,Singaporean,1991-11-27,Degree,Bachelor Science (Facilities & Events Manageme...,2017-06-10,HR Administrator,14th,...,Subsidiesed,107,NETS,1212,2.5,L,456,1101,32,2 - 3
4,2020/1101-014/005,Ms,F,Singaporean,1985-04-23,Diploma,"Diploma in Procurement and Supply Management, ...",2016-08-10,Purchasing Executive,14th,...,Subsidiesed,107,NETS,1712,2.4,L,456,1101,39,2 - 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2019/5113-007/005,Ms,F,Singaporean,1973-01-10,Degree,Bachelor of Commerce (Accounting and Finance)/...,2017-11-05,"Head, Registry & Corp Admin",7th,...,Individual,107,Bank Transfer,5803,2.9,L,183,5113,51,2 - 3
240,2019/5113-007/006,Ms,F,Permanent Resident,1982-09-30,Degree,Bachelor of Economics (Accounting)/\r\nUnivers...,2016-05-05,Finance Officer,7th,...,Individual,107,Bank Transfer,5803,2.0,L,183,5113,41,2 - 3
241,2019/5113-006/003,Ms,F,Singaporean,1990-06-03,Degree,Bachelor of Science in Hotel Administration (H...,2015-06-08,HR Assistant,6th,...,Individual,107,Bank Transfer,5803,2.8,L,183,5113,34,2 - 3
242,2019/5113-005/001,Ms,F,Singaporean,1981-11-14,Degree,Bachelor of Arts with Second Class Honours (Lo...,2014-08-04,Household Goods Coordinator,5th,...,Individual,107,Bank Transfer,5803,1.9,L,180,5113,42,1 - 2


In [60]:
import pandas as pd
import re

def categorize_job_title(title):
    if pd.isna(title):
        return 'Not Specified'
    
    title = str(title).lower()
    
    if re.search(r'hr|human resource|recruit|talent|people ops', title):
        return 'Human Resources'
    elif re.search(r'admin|secretary|receptionist|clerk|office', title):
        return 'Administration'
    elif re.search(r'manager|director|head|lead|supervisor|vice president', title):
        return 'Management'
    elif re.search(r'account|finance|payroll', title):
        return 'Finance and Accounting'
    elif re.search(r'client|sales|service|customer', title):
        return 'Customer Service and Sales'
    elif re.search(r'operation|logistics|supply|coordinator', title):
        return 'Operations and Logistics'
    elif re.search(r'market|pr|public relation|brand|communication', title):
        return 'Marketing and Public Relations'
    elif re.search(r'teacher|lecturer|trainer|education|learning', title):
        return 'Education and Training'
    elif re.search(r'it|tech|developer|engineer|analyst', title):
        return 'Technical and IT'
    else:
        return 'Others'

def add_job_category(df):
    df['Job Category'] = df['DESIGNATION'].apply(categorize_job_title)
    return df

# Assuming your dataframe is called 'df'
df = add_job_category(df)

# Plotly Sunburst

In [61]:
# Prepare the data for the sunburst plot
# Create simplified age groups
df['Age Group'] = pd.cut(df['Age'], bins=[18, 35, 60], labels=['Millenial', 'Middle Aged'])

# Drop rows with missing values in the relevant columns
df.dropna(subset=['COURSE', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['COURSE'].value_counts().nlargest(5).index
df_filtered = df[df['COURSE'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['COURSE', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each level
course_gpa = df_filtered.groupby('COURSE')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='COURSE')

# Create the sunburst plot
fig = px.sunburst(df_grouped, path=['COURSE', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update hover template to show average GPAs for each level
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}'
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.975,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=40, l=0, r=0, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Show the plot
fig.show()


In [62]:
import pandas as pd
import plotly.express as px
import numpy as np

# Assuming df is your DataFrame
# If not, you need to load your data into df first

# Create simplified age groups
df['Age Group'] = pd.cut(df['Age'], bins=[18, 35, 60], labels=['Millenial', 'Middle Aged'])

# Drop rows with missing values in the relevant columns
df.dropna(subset=['HIGHEST QUALIFICATION', 'Age Group', 'GPA'], inplace=True)

# Filter data to include only the top 5 courses by student count
top_courses = df['HIGHEST QUALIFICATION'].value_counts().nlargest(6).index
df_filtered = df[df['HIGHEST QUALIFICATION'].isin(top_courses)]

# Add a Count column for aggregation
df_filtered['Count'] = 1

# Group by course and age group and calculate average GPA and count
df_grouped = df_filtered.groupby(['HIGHEST QUALIFICATION', 'Age Group']).agg(
    Count=('Count', 'sum'),
    Avg_GPA=('GPA', 'mean')
).reset_index()

# Calculate average GPA for each level
course_gpa = df_filtered.groupby('HIGHEST QUALIFICATION')['GPA'].mean().reset_index(name='GPA_course')

# Merge the average GPAs into the grouped dataframe
df_grouped = df_grouped.merge(course_gpa, on='HIGHEST QUALIFICATION')

# Filter out any potential zero counts
df_grouped = df_grouped[df_grouped['Count'] > 0]

# Create the sunburst plot
fig = px.sunburst(df_grouped, path=['HIGHEST QUALIFICATION', 'Age Group'], values='Count',
                  color='Avg_GPA', 
                  hover_data={
                      'Avg_GPA': ':.2f',
                      'GPA_course': ':.2f'
                  })

# Update hover template to show average GPAs for each level
fig.update_traces(
    hovertemplate='<b>%{label}</b><br>' +
                  'Count: %{value}<br>' +
                  'Course GPA: %{customdata[1]:.2f}<br>' +
                  'Age Group GPA: %{customdata[0]:.2f}'
)

# Update layout for better aesthetics and readability
fig.update_layout(
    title={
        'text': 'Course and Age with Average GPAs',
        'y':0.975,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='black'),
    font=dict(family='Arial, sans-serif', size=16, color='black'),
    margin=dict(t=40, l=0, r=0, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Show the plot
fig.show()

In [63]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

colors = {
    'background': '#111111',
    'text': '#ffffff'
}

df['COURSE'] = df['COURSE'].astype(str)

# Create initial figures
fig_bar = px.bar(df, x="COURSE", barmode="group")
fig_pie_1 = px.pie(df, values='GPA', names='COURSE', title='Course Distribution')
fig_pie_2 = px.pie(df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

# Update layout for all figures
for fig in [fig_bar, fig_pie_1, fig_pie_2]:
    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )

Age_Range = df['Age'].unique()
Age_Range.sort()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("DAVI CA2 Dashboard", className="text-center mb-4 mt-4"), width=12)
    ]),

    dbc.Row([
        # Sidebar with radio buttons
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H3("Filters", className="text-center")),
                dbc.CardBody([
                    html.H6("Job Category"),
                    dcc.Dropdown(id='Job-category', options=[{'label': 'All Data', 'value': 'ALL'}]+[{'label': country, 'value': country}
                          for country in df['Job Category'].unique()],style={'color': 'black'}, value='ALL',multi=True),
                    html.H6("Nationality", className='mt-4'),
                    dcc.RadioItems(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + 
                                [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        className="mb-2"
                    ),
                    html.H6("Age Range"),
                    dcc.RangeSlider(Age_Range.min(), Age_Range.max(), 3, value=[Age_Range.min(), Age_Range.max()], id='my-range-slider'),
                    html.H6("Male or Female"),
                    daq.BooleanSwitch(
                    on=True,
                    color="#9B51E0",
                    )

                ])
            ])
        ], width=3),

        # Main content area
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='example-graph-1', figure=fig_bar)
                ])
            ], className="mb-4"),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-1', figure=fig_pie_1)
                        ])
                    ])
                ], width=6),
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            dcc.Graph(id='example-pie-2', figure=fig_pie_2)
                        ])
                    ])
                ], width=6)
            ])
        ], width=9)
    ])
], fluid=True, style={'background-color': 'eggshell'})

# Define callback to update graphs
@app.callback(
    Output('example-graph-1', 'figure'),
    Output('example-pie-1', 'figure'),
    Output('example-pie-2', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value')
)

def update_graphs(selected_nationality, selected_job_categories):
    # If 'ALL' is selected in either dropdown, it means no filtering on that dimension
    if 'ALL' in selected_nationality:
        filtered_df = df
    else:
        filtered_df = df[df['NATIONALITY'].isin(selected_nationality)]
    
    if 'ALL' not in selected_job_categories:
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
    
    fig_bar = px.bar(filtered_df, x="COURSE", y="GPA", barmode="group",
                     title="GPA Distribution by Course and Gender")
    fig_pie_1 = px.pie(filtered_df, values='GPA', names='COURSE', title='Course Distribution')
    fig_pie_2 = px.pie(filtered_df, values='GPA', names='NATIONALITY', title='Nationality Distribution')

    for fig in [fig_bar, fig_pie_1, fig_pie_2]:
        fig.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text']
        )

    return fig_bar, fig_pie_1, fig_pie_2
    
if __name__ == '__main__':
    app.run_server(debug=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242 entries, 0 to 243
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   STUDENT ID                             242 non-null    object  
 1   SALUTATION                             242 non-null    object  
 2   GENDER                                 242 non-null    object  
 3   NATIONALITY                            242 non-null    object  
 4   DOB                                    242 non-null    object  
 5   HIGHEST QUALIFICATION                  242 non-null    object  
 6   NAME OF QUALIFICATION AND INSTITUTION  242 non-null    object  
 7   DATE ATTAINED HIGHEST QUALIFICATION    242 non-null    object  
 8   DESIGNATION                            242 non-null    object  
 9   INTAKE NO                              242 non-null    object  
 10  COMMENCEMENT DATE                      242 non-null    object  
 11

In [65]:
df.DESIGNATION.unique()

array(['Snr Associate, Client Services', 'Admin Officer',
       'Admin Assistant', 'HR Administrator', 'Purchasing Executive',
       'HR Executive', 'PR Executive', 'No Designation',
       'Accounts cum HR Executive', 'Admin & HR Executive',
       'HR Assistant', 'Rooms Division Manager',
       'HR Associate, Southeast Asia', 'Administrative Executive',
       'Admin & HR Assistant', 'HR Support / Office Manager',
       'Executive, Administration', 'Administrator',
       'Admin & Finance Executive', 'Operation Executive', 'Clerk',
       'HR & Admin', 'Early Childhood Support Office', 'Office Manager',
       'Business Development Executive', 'Human Resource Assistant',
       'Executive, Planning & Delivery', 'Secretary',
       'Executive Assistant', 'Senior Assistant (Tenant Administration)',
       'Senior HR Officer', 'Manager',
       'Relationship and Marketing Executive', 'Administrative Assistant',
       'Business Support Manager, APAC', 'Team Assistant',
       'APAC 

In [66]:
df[df['DESIGNATION'] == 'No Designation']

,STUDENT ID,SALUTATION,GENDER,NATIONALITY,DOB,HIGHEST QUALIFICATION,NAME OF QUALIFICATION AND INSTITUTION,DATE ATTAINED HIGHEST QUALIFICATION,DESIGNATION,INTAKE NO,...,PAYMENT MODE,COURSE FEE,GPA,CITIZENSHIP_STATUS,date_diff,COURSE,Age,GPA Category,Job Category,Age Group
8,2020/1101-015/010,Ms,F,Singaporean,1977-09-06,Degree,"Bachelor of Business (Advertising), Queensland...",2018-10-06,No Designation,15th,...,GIRO,1712,2.2,L,456,1101,46,2 - 3,Others,Middle Aged
9,2020/1101-015/011,Ms,F,Singaporean,1991-05-12,Diploma,"Diploma in Environmental Science, Republic Pol...",2016-09-06,No Designation,15th,...,GIRO,1712,3.1,L,456,1101,33,3 - 3.5,Others,Millenial
11,2020/1101-016/013,Ms,F,Singaporean,1989-02-10,Degree,"Bachelor of Science (Business Management), Uni...",2019-08-01,No Designation,16th,...,GIRO,1712,4.0,L,455,1101,35,More Than 3.5,Others,Millenial
16,2020/1101-017/018,Ms,F,Singaporean,1983-06-18,Certificate,N level,2018-01-02,No Designation,17th,...,GIRO,1712,3.3,L,457,1101,41,3 - 3.5,Others,Middle Aged
20,2019/1101-007/003,Ms,F,Foreigner,1990-01-02,Degree,"Bachelor of Business Administration, Universit...",2015-08-08,No Designation,7th,...,NETS,1712,3.1,F,455,1101,34,3 - 3.5,Others,Millenial
24,2019/1101-008/007,Ms,F,Permanent Resident,1979-10-22,Master,"Master in Computer Science, Osmania University...",2015-06-07,No Designation,8th,...,Bank Transfer,1712,2.7,F,457,1101,44,2 - 3,Others,Middle Aged
45,2019/1101-011/029,Ms,F,Singaporean,1973-04-30,Certificate,"Certified Accounting Technicians (CATS), ACCA,...",2016-09-10,No Designation,11th,...,Bank Transfer,1712,2.9,L,456,1101,51,2 - 3,Others,Middle Aged
47,2019/1101-012/031,Ms,F,Singaporean,1976-04-27,Certificate,"O' level, 'N' Level, WPLN",2016-08-05,No Designation,12th,...,NETS,1712,3.1,L,457,1101,48,3 - 3.5,Others,Middle Aged
53,2020/2101-111/004,Ms,F,Singaporean,1992-04-18,Certificate,"Foundation Certificate in HRM, SHRI Academy",2019-01-28,No Designation,111th,...,NETS,2996,2.4,L,268,2101,32,2 - 3,Others,Millenial
68,2019/2101-107/007,Ms,F,Singaporean,1999-05-24,Certificate,O' levels,2018-04-24,No Designation,107th,...,GIRO,2996,3.0,L,274,2101,25,3 - 3.5,Others,Millenial


In [67]:
df['Age Group'] = pd.cut(df['Age'], bins=[0, 25, 40, np.inf], labels=['Below 25', '25 to 40', '40 and Above'])

In [113]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

colors = {
    'background': '#f8f9fa',
    'text': '#2c3e50',
    'primary': '#3498db',
    'secondary': '#2ecc71',
    'accent': '#e74c3c'
}

# Assume df is your DataFrame
# Make sure to load your data into df before this point

def create_heatmap(df, x_axis, y_axis, title_suffix=''):
    heatmap_data = pd.crosstab(df[y_axis], df[x_axis])
    
    fig_heatmap = go.Figure(data=go.Heatmap(
                   z=heatmap_data.values,
                   x=heatmap_data.columns,
                   y=heatmap_data.index,
                   colorscale='Blues',
                   text=heatmap_data.values,
                   texttemplate='%{text}',
                   colorbar=dict(title='Count')
    ))

    fig_heatmap.update_layout(
        title={
            'text': f'{y_axis} vs {x_axis} {title_suffix}',
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title=x_axis,
        yaxis_title=y_axis,
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        title_font_size=20,
        legend_title_font_size=14,
        legend_font_size=12,
        margin=dict(l=40, r=40, t=40, b=40), height=400
    )
    
    return fig_heatmap

def create_age_distribution_pie(df):
    age_counts = df['Age Group'].value_counts()
    fig = go.Figure(data=[go.Pie(
        labels=age_counts.index, 
        values=age_counts.values, 
        hole=0.35,
        textposition='inside',
        insidetextorientation='radial'
    )])
    fig.update_layout(
        legend=dict(
            orientation="h",
            y=-0.01,
            x=0.5
        ),
        margin=dict(l=20, r=20, t=2, b=2),
        autosize=True
    )
    return fig

age_groups = df['HIGHEST QUALIFICATION'].unique()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_columns = [col for col in categorical_columns if col not in [
    'STUDENT ID', 'NAME OF QUALIFICATION AND INSTITUTION', 'DATE ATTAINED HIGHEST QUALIFICATION',
    'DESIGNATION', 'INTAKE NO', 'COMMENCEMENT DATE', 'FULL-TIME OR PART-TIME', 'COURSE FUNDING',
    'PAYMENT MODE', 'DOB', 'COMPLETION DATE', 'SALUTATION', 'HIGHEST QUALIFICATION'
]]

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Rejey DAVI Dashboard", className="display-4 text-primary mb-4")
        ], width=12, className='text-center')
    ]),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Filters", className="card-title mb-4", style={'text-align': 'center','text-decoration': 'underline'}),
                    html.H6("X axis", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='x-axis',
                        options=[{'label': cat, 'value': cat} for cat in categorical_columns],
                        value='Age Group',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Y axis", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='y-axis',
                        options=[{'label': cat, 'value': cat} for cat in categorical_columns],
                        value='Job Category',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Job Category", className="text-muted"),
                    dcc.Dropdown(
                        id='Job-category',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': category, 'value': category} for category in df['Job Category'].unique()],
                        value='ALL',
                        multi=True,
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Nationality", className='text-muted mt-4'),
                    dcc.Dropdown(
                        id='radio-buttons',
                        options=[{'label': 'All Data', 'value': 'ALL'}] + [{'label': nat, 'value': nat} for nat in df['NATIONALITY'].unique()],
                        value='ALL',
                        style={'color': colors['text']},
                        className="mb-3"
                    ),
                    html.H6("Gender", className='text-muted mt-4'),
                    dbc.RadioItems(
                        id='Gender',
                        options=[
                            {'label': 'All', 'value': 'ALL'},
                            {'label': 'Male', 'value': 'M'},
                            {'label': 'Female', 'value': 'F'}
                        ],
                        value='ALL',
                        inline=True,
                        className="mb-3"
                    )
                ])
            ], className="shadow-sm", style={'border-radius': '10px','height':'95.5%'})
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Tabs(id='heatmap-tabs', value=age_groups[0], children=[
                        dcc.Tab(label=group, value=group) for group in age_groups
                    ]),
                    html.Div(id='heatmap-container')
                ])
            ], className="mb-4"),
        ], width=9)
    ]),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Summary", className="card-title mb-4", style={'text-align': 'center', 'text-decoration': 'underline','font-weight':'bold'}),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Total Students", className="card-title mb-2", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='Total_Students', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    html.Hr(),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Overall GPA", className="card-title mt-2 mb-2", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='Overall_Gpa',className='mt-2', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    html.Hr(),
                    dbc.Row(
                        [
                            dbc.Col(html.H5("Course", className="card-title mt-2 mb-4", style={'text-align': 'left','font-weight':'bold'}), width="auto"),
                            dbc.Col(html.H5("Average GPA", className="card-title mt-2 mb-4", style={'text-align': 'right','font-weight':'bold'}), width="auto")
                        ],
                        className="d-flex justify-content-between"
                    ),
                    dbc.Row(
                        [
                            dbc.Col(html.H6('Diploma in Business Administration', style={'text-align': 'left'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='1-gpa', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    dbc.Row(
                        [
                            dbc.Col(html.H6('Certificate in HR Management', style={'text-align': 'left'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='2-gpa', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    dbc.Row(
                        [
                            dbc.Col(html.H6('Certificate in Digital Marketing', style={'text-align': 'left'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='3-gpa', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    dbc.Row(
                        [
                            dbc.Col(html.H6('Bachelor of Business Administration', style={'text-align': 'left'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='4-gpa', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    ),
                    dbc.Row(
                        [
                            dbc.Col(html.H6('Master of Business Administration', style={'text-align': 'left'}), width="auto"),
                            dbc.Col(html.H6(
                                html.H4(id='5-gpa', style={'text-align': 'right'})
                            ), width="auto", style={'text-align': 'right'})
                        ],
                        className="d-flex justify-content-between"
                    )
                ])
            ], className="mb-4")
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Age Distribution", className="card-title"),
                    dcc.Graph(id='age-distribution-pie', config={'displayModeBar': False})
                ])
            ], className="", style={'height': '94%'})
        ], width=3),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Nationality Distribution", className="card-title"),
                ])
            ], className="mb-4")
        ], width=6)    ])
], fluid=True)

@app.callback(
    Output('heatmap-container', 'children'),
    Output('1-gpa', 'children'),
    Output('2-gpa', 'children'),
    Output('3-gpa', 'children'),
    Output('4-gpa', 'children'),
    Output('5-gpa', 'children'),
    Output('Total_Students', 'children'),
    Output('Overall_Gpa', 'children'),
    Output('age-distribution-pie', 'figure'),
    Input('radio-buttons', 'value'),
    Input('Job-category', 'value'),
    Input('Gender', 'value'),
    Input('heatmap-tabs', 'value'),
    Input('x-axis', 'value'),
    Input('y-axis', 'value')
)

def update_graphs(selected_nationality, selected_job_categories, selected_gender, selected_tab, x_axis, y_axis):
    filtered_df = df

    if selected_nationality != 'ALL':
        filtered_df = filtered_df[filtered_df['NATIONALITY'] == selected_nationality]
    
    if 'ALL' not in selected_job_categories:
        filtered_df = filtered_df[filtered_df['Job Category'].isin(selected_job_categories)]
    
    filtered_df = filtered_df[filtered_df['HIGHEST QUALIFICATION'] == selected_tab]
    
    if selected_gender != 'ALL':
        filtered_df = filtered_df[filtered_df['GENDER'] == selected_gender]
    
    fig_heatmap = create_heatmap(filtered_df, x_axis, y_axis, f'(Age Group {selected_tab})')
    fig_age_distribution = create_age_distribution_pie(filtered_df)

    a = round(filtered_df.loc[filtered_df['COURSE'] == '1101', 'GPA'].mean(), 2)
    b = round(filtered_df.loc[filtered_df['COURSE'] == '2101', 'GPA'].mean(), 2)
    c = round(filtered_df.loc[filtered_df['COURSE'] == '2102', 'GPA'].mean(), 2)
    d = round(filtered_df.loc[filtered_df['COURSE'] == '5112', 'GPA'].mean(), 2)
    e = round(filtered_df.loc[filtered_df['COURSE'] == '5113', 'GPA'].mean(), 2)

    a = a if pd.notna(a) else "N/A"
    b = b if pd.notna(b) else "N/A"
    c = c if pd.notna(c) else "N/A"
    d = d if pd.notna(d) else "N/A"
    e = e if pd.notna(e) else "N/A"

    Total_Students = filtered_df.shape[0]
    
    Overall_Gpa = round(filtered_df['GPA'].mean(), 2)
    
    return dcc.Graph(id='heatmap-graph', figure=fig_heatmap), a, b, c, d, e, Total_Students, Overall_Gpa, fig_age_distribution

if __name__ == '__main__':
    app.run_server(debug=True)